<a href="https://colab.research.google.com/github/moebot3000/SpecialistCode/blob/main/BBS_scrape.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# BBS licensure records

This program can be used to get data from the BBS website. It looks up each student who has graduated from our program in a spreadsheet to find out if they registered as an Associate and if they got licensed by the BBS.

*Get libraries that will be used in the analysis. These are just mini-programs.*

In [ ]:
from lxml import html
import requests
from bs4 import BeautifulSoup
import os
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import re
from math import isnan
import scipy as sp
from datetime import datetime  
from datetime import timedelta
from google.colab import files
!pip install --upgrade --quiet gspread
from google.colab import auth
auth.authenticate_user()

import gspread
from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())
from gspread_dataframe import get_as_dataframe
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


*Next, upload an excel spreadsheet (saved as a .csv) of all the data from FileMaker. This will be used to get a list of student names. Here it's coming from my GitHub Ed account, but it can also come from Google Drive or wherever.*

*Below, you can see the first few rows of the spreadsheet.*

In [ ]:
url = 'https://raw.githubusercontent.com/moebot3000/Annual_Report_Self_Study_2019_wi/master/FM_10092019_background.csv?token=AL3PAHR2JDGKL7RNGTBYYOS5VTRFI'
FM = pd.read_csv(url)
FM.head()

,StudentDataSystem::LogoSquare,StudentDataSystem::ButtonNav2,StudentDataSystem::ButtonNav1,StudentDataSystem::ButtonNav3,StudentDataSystem::ButtonNav4,StudentDataSystem::ButtonNav5,StudentDataSystem::gReportStudentBasic,StudentDataSystem::ButtonGreyPlus,StudentDataSystem::TextNav2,StudentDataSystem::TextNav1,StudentDataSystem::TextNav3,StudentDataSystem::TextNav4,StudentDataSystem::TextNav5,Photos::Photo,NameLast,NameFirst,NameInformal,NameTitle,NameMiddle,NameSuffix,Sem_Year_Admitted,Cohort,ID_Unique,MailAddr::z.MailingAddress,z.PrimaryPhone,DateOfBirth,Gender,AcademicStanding,DateGrad,ID_Advisor,Instructors|Advisors::NameFull,<Table Missing>,peer_mentor,Status_Most_Recent,SAStatus::DateBegan,SAStatus::DateComplete,Numbers|PrimaryEmail::MaskDescription,StudentDataSystem::ButtonFind,StudentDataSystem::TextFind,StudentDataSystem::ButtonFindExtend,...,Numbers::Type,Numbers::Location,Numbers::MaskDescription,Numbers::Description,Numbers::Extension,Numbers::DoNotPrint,Numbers::Note,Numbers::isPrimary,AgeYears,AgeYears_at_admit,StudentDataSystem::ButtonNewRecordSmall.1,Addresses::Location,Addresses::StreetLine1,Addresses::City,Addresses::State,Addresses::PostalCode,Addresses::Country,Addresses::DoNotPrint,Addresses::Addr_Valid_to_date,Addresses::isPermanent,Addresses::isMailing,StudentDataSystem::ButtonDeleteRecord.2,Addresses::StreetLine2,StudentDataSystem::ButtonNewRecordSmall.2,Language,Alt_Contact_Name,Ethnic_name1,NotesGeneral,Birthplace_Cntry_Name,Alt_Contact_addr,EthnicitySpecified,Citizenship,CountSOC,Visa_Name,Alt_Contact_phone,NameFormerLast,DateCreated,AccountCreatedBy,DateModified,AccountModifiedBy
0,?,woman3_32x32.png,users_32x32.png,briefcase_doc_24x24.png,books_24x24.png,note_edit_24x24.png,NaN,?,Enrollment,Faculty,Agencies,Courses,Sections,NaN,Berman,Cathy,NaN,NaN,NaN,NaN,Fall 2006,NaN,595.0,"2629 Ashby Ave Apt B\nBerkeley, CA 94705",510-798-6570,10/25/1956,F,NaN,10/19/2008,5.0,Milena Esherick,NaN,NaN,Graduated,NaN,10/01/2008,Cberman@wi.edu,advancedfind.png,Find,find_add_32x32.png,...,Phone,Home,510-798-6570,510-798-6570,NaN,NaN,NaN,1.0,62.0,49.0,down_plus_24x24.png,Home,2629 Ashby Ave Apt B,Berkeley,CA,94705,USA,NaN,NaN,NaN,1.0,cancel.png,NaN,down_plus_24x24.png,NaN,NaN,4 Euro Am / Mid Eastern Am / White,\n\n\n\n\n\n\n,NaN,NaN,NaN,American,0.0,NaN,NaN,NaN,09/16/2002,admin,08/21/2017,Eugene
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Email,NaN,Cberman@wi.edu,Cberman@wi.edu,NaN,NaN,NaN,1.0,NaN,NaN,NaN,Home,NaN,NaN,NaN,NaN,USA,NaN,NaN,NaN,NaN,cancel.png,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Phone,Work,510-620-5667,510-620-5667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Email,NaN,onesteptherapy@gmail.com,onesteptherapy@gmail.com,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,?,woman3_32x32.png,users_32x32.png,briefcase_doc_24x24.png,books_24x24.png,note_edit_24x24.png,NaN,?,Enrollment,Faculty,Agencies,Courses,Sections,NaN,Lankford,Ashby,NaN,NaN,NaN,NaN,Fall 2006,NaN,596.0,"1606 Delaware Street\nBerkeley, CA 94704",510-220-4441,08/12/1981,F,NaN,10/01/2008,5.0,Milena Esherick,NaN,NaN,Graduated,NaN,10/01/2008,ALankford@wi.edu,advancedfind.png,Find,find_add_32x32.png,...,Phone,Home,510-220-4441,510-220-4441,NaN,NaN,NaN,1.0,38.0,25.0,down_plus_24x24.png,Home,1606 Delaware Street,Berkeley,CA,94704,USA,NaN,NaN,NaN,1.0,cancel.png,NaN,down_plus_24x24.pn

*Clean up that spreadsheet and pull out just the parts used in this analysis. Again, you can see the top of the cleaned up version below.*

In [ ]:
stus = FM.filter(['NameLast','NameFirst','Sem_Year_Admitted','Status_Most_Recent'])
stus.dropna(how='all',inplace=True)
stus['entered'] = stus.Sem_Year_Admitted.str.extract('(\d+)')
stus.reset_index(inplace=True)
stus['NameLast'] = stus['NameLast'].str.upper()
stus['NameFirst'] = stus['NameFirst'].str.upper()
stus = stus[stus['Status_Most_Recent'] == 'Graduated']
stus.head()

,index,NameLast,NameFirst,Sem_Year_Admitted,Status_Most_Recent,entered
0,0,BERMAN,CATHY,Fall 2006,Graduated,2006
1,4,LANKFORD,ASHBY,Fall 2006,Graduated,2006
2,8,ORTIZ,LAURA,Fall 2006,Graduated,2006
3,13,MOK,IRVING,Fall 2006,Graduated,2006
4,17,KONIGSBERG,LAURA,Fall 2006,Graduated,2006


*Use those names to access the BBS website to look everybody up and see what kind of licenses they've got/when they were issued. That info gets added to the spreadsheet.*

In [ ]:
mft_first_date = {}
mft_first_type = {}
mft_first_num = {}
for i,j in zip(stus['NameLast'], stus['NameFirst']):
    body = {'lastName':i, 'firstName':j, 'licenseType':'217'}
    con = requests.post('https://search.dca.ca.gov/results', data=body)
    s1 = con.text
    s2 = i+', '+j
    placetocut = s1.find('/de',s1.find(s2))
    cuttext = s1[placetocut:]
    placetocut_lictype = cuttext.find('</strong>', cuttext.find('class'))
    cuttext_lictype = cuttext[placetocut_lictype:]
    lic_type = cuttext_lictype[9:18]
    endgone = cuttext.rsplit(sep='\' cl')
    linkstring=endgone[0]
    newlink= 'https://search.dca.ca.gov'+linkstring
    try:
        con2 = requests.get(newlink)
        s3=con2.text
        placetocut2 = s3.find('ate">')
        cuttext2 = s3[placetocut2:]
        endgone2 = cuttext2.rsplit(sep='</p>')
        messyissue=endgone2[0]
        firstdate = messyissue.replace('ate">','')
        placetocut_licnum = s3.find('Licensing details for: ')
        cuttext_licnum = s3[placetocut_licnum:]
        lic_num = re.search(r'\d+', cuttext_licnum).group()
        mft_first_date.update({s2 : firstdate})
        mft_first_type.update({s2 : lic_type})
        mft_first_num.update({s2: lic_num})
    except requests.ConnectionError:
        mft_first_date.update({s2 : 'NaN'})
        mft_first_type.update({s2 : 'NaN'})
        mft_first_num.update({s2: 'NaN'})

mft_second_date = {}
mft_second_type = {}
mft_second_num = {}
for i,j in zip(stus['NameLast'], stus['NameFirst']):
    body = {'lastName':i, 'firstName':j, 'licenseType':'217'}
    con = requests.post('https://search.dca.ca.gov/results', data=body)
    s1 = con.text
    s2 = i+', '+j
    placetocut = s1.find('/de',s1.find(s2))
    cuttext = s1[placetocut:]
    placetocut2 = cuttext.find('/de',cuttext.find(s2))
    cuttext2 = cuttext[placetocut2:]
    placetocut_lictype = cuttext2.find('</strong>', cuttext2.find('class'))
    cuttext_lictype = cuttext2[placetocut_lictype:]
    lic_type = cuttext_lictype[9:18]
    endgone = cuttext2.rsplit(sep='\' cl')
    linkstring=endgone[0]
    newlink= 'https://search.dca.ca.gov'+linkstring
    try:
        con2 = requests.get(newlink)
        s3=con2.text
        placetocut2 = s3.find('ate">')
        cuttext2 = s3[placetocut2:]
        endgone2 = cuttext2.rsplit(sep='</p>')
        messyissue=endgone2[0]
        seconddate = messyissue.replace('ate">','')
        placetocut_licnum = s3.find('Licensing details for: ')
        cuttext_licnum = s3[placetocut_licnum:]
        lic_num = re.search(r'\d+', cuttext_licnum).group()
        mft_second_date.update({s2 : seconddate})
        mft_second_type.update({s2 : lic_type})
        mft_second_num.update({s2: lic_num})
    except requests.ConnectionError:
        mft_second_date.update({s2 : 'NaN'})
        mft_second_type.update({s2 : 'NaN'})
        mft_second_num.update({s2: 'NaN'})

stus['comboname'] = stus['NameLast']+', '+stus['NameFirst']
mftseconddatename = pd.DataFrame.from_dict(mft_second_date, orient='index', columns=['mft_second_date'])
mftseconddatetype = pd.DataFrame.from_dict(mft_second_type, orient='index', columns=['mft_second_type'])
mftseconddatenum = pd.DataFrame.from_dict(mft_second_num, orient='index', columns=['mft_second_num'])
stusecond = stus.join(mftseconddatename, on='comboname', how='left', lsuffix='_left', rsuffix='_right')
stusecondtype = stusecond.join(mftseconddatetype, on='comboname', how='left', lsuffix='_left', rsuffix='_right')
stusecondtypes = stusecondtype.join(mftseconddatenum, on='comboname', how='left', lsuffix='_left', rsuffix='_right')

mftfirstdatename = pd.DataFrame.from_dict(mft_first_date, orient='index', columns=['mft_first_date'])
mftfirstdatetype = pd.DataFrame.from_dict(mft_first_type, orient='index', columns=['mft_first_type'])
mftfirstdatenum = pd.DataFrame.from_dict(mft_first_num, orient='index', columns=['mft_first_num'])
stusecondtype_first = stusecondtypes.join(mftfirstdatename, on='comboname', how='left', lsuffix='_left', rsuffix='_right')
stu_secondtype_firsttype = stusecondtype_first.join(mftfirstdatetype, on='comboname', how='left', lsuffix='_left', rsuffix='_right')
stu_secondtype_firsttypes = stu_secondtype_firsttype.join(mftfirstdatenum, on='comboname', how='left', lsuffix='_left', rsuffix='_right')

stu_secondtype_firsttypes.head()

todaysdate = datetime.today().strftime('%m%d%Y')
output = 'stu_secondtype_firsttypes_' + todaysdate + '.csv'
stu_secondtype_firsttypes.to_csv(output)
files.download(output)

MessageError: ignored

## RUN FROM HERE 09242019

*Do the same thing, but for PCC licensure this time.*

In [ ]:
pcc_first_date = {}
pcc_first_type = {}
pcc_first_num = {}
for i,j in zip(stus['NameLast'], stus['NameFirst']):
    body = {'lastName':i, 'firstName':j, 'licenseType':'221'}
    con = requests.post('https://search.dca.ca.gov/results', data=body)
    s1 = con.text
    s2 = i+', '+j
    placetocut = s1.find('/de',s1.find(s2))
    cuttext = s1[placetocut:]
    placetocut_lictype = cuttext.find('</strong>', cuttext.find('class'))
    cuttext_lictype = cuttext[placetocut_lictype:]
    lic_type = cuttext_lictype[9:18]
    endgone = cuttext.rsplit(sep='\' cl')
    linkstring=endgone[0]
    newlink= 'https://search.dca.ca.gov'+linkstring
    try:
        con2 = requests.get(newlink)
        s3=con2.text
        placetocut2 = s3.find('ate">')
        cuttext2 = s3[placetocut2:]
        endgone2 = cuttext2.rsplit(sep='</p>')
        messyissue=endgone2[0]
        firstdate = messyissue.replace('ate">','')
        placetocut_licnum = s3.find('Licensing details for: ')
        cuttext_licnum = s3[placetocut_licnum:]
        lic_num = re.search(r'\d+', cuttext_licnum).group()
        pcc_first_date.update({s2 : firstdate})
        pcc_first_type.update({s2 : lic_type})
        pcc_first_num.update({s2: lic_num})
    except requests.ConnectionError:
        pcc_first_date.update({s2 : 'NaN'})
        pcc_first_type.update({s2 : 'NaN'})
        pcc_first_num.update({s2: 'NaN'})

pcc_second_date = {}
pcc_second_type = {}
pcc_second_num = {}
for i,j in zip(stus['NameLast'], stus['NameFirst']):
    body = {'lastName':i, 'firstName':j, 'licenseType':'221'}
    con = requests.post('https://search.dca.ca.gov/results', data=body)
    s1 = con.text
    s2 = i+', '+j
    placetocut = s1.find('/de',s1.find(s2))
    cuttext = s1[placetocut:]
    placetocut2 = cuttext.find('/de',cuttext.find(s2))
    cuttext2 = cuttext[placetocut2:]
    placetocut_lictype = cuttext2.find('</strong>', cuttext2.find('class'))
    cuttext_lictype = cuttext2[placetocut_lictype:]
    lic_type = cuttext_lictype[9:18]
    endgone = cuttext2.rsplit(sep='\' cl')
    linkstring=endgone[0]
    newlink= 'https://search.dca.ca.gov'+linkstring
    try:
        con2 = requests.get(newlink)
        s3=con2.text
        placetocut2 = s3.find('ate">')
        cuttext2 = s3[placetocut2:]
        endgone2 = cuttext2.rsplit(sep='</p>')
        messyissue=endgone2[0]
        seconddate = messyissue.replace('ate">','')
        placetocut_licnum = s3.find('Licensing details for: ')
        cuttext_licnum = s3[placetocut_licnum:]
        lic_num = re.search(r'\d+', cuttext_licnum).group()
        pcc_second_date.update({s2 : seconddate})
        pcc_second_type.update({s2 : lic_type})
        pcc_second_num.update({s2: lic_num})
    except requests.ConnectionError:
        pcc_second_date.update({s2 : 'NaN'})
        pcc_second_type.update({s2 : 'NaN'})
        pcc_second_num.update({s2: 'NaN'})

pccseconddatename = pd.DataFrame.from_dict(pcc_second_date, orient='index', columns=['pcc_second_date'])
pccseconddatetype = pd.DataFrame.from_dict(pcc_second_type, orient='index', columns=['pcc_second_type'])
pccseconddatenum = pd.DataFrame.from_dict(pcc_second_num, orient='index', columns=['pcc_second_num'])
stu2second = stu_secondtype_firsttypes.join(pccseconddatename, on='comboname', how='left', lsuffix='_left', rsuffix='_right')
stu2secondtype = stu2second.join(pccseconddatetype, on='comboname', how='left', lsuffix='_left', rsuffix='_right')
stu2secondtypes = stu2second.join(pccseconddatenum, on='comboname', how='left', lsuffix='_left', rsuffix='_right')

pccfirstdatename = pd.DataFrame.from_dict(pcc_first_date, orient='index', columns=['pcc_first_date'])
pccfirstdatetype = pd.DataFrame.from_dict(pcc_first_type, orient='index', columns=['pcc_first_type'])
pccfirstdatenum = pd.DataFrame.from_dict(pcc_first_num, orient='index', columns=['pcc_first_num'])
stu2secondtype_first = stu2secondtypes.join(pccfirstdatename, on='comboname', how='left', lsuffix='_left', rsuffix='_right')
stu2_secondtype_firsttype = stu2secondtype_first.join(pccfirstdatetype, on='comboname', how='left', lsuffix='_left', rsuffix='_right')
stu2_secondtype_firsttypes = stu2_secondtype_firsttype.join(pccfirstdatenum, on='comboname', how='left', lsuffix='_left', rsuffix='_right')

stu2_secondtype_firsttypes.head()

spreadsheet = 'stu2_secondtype_firsttypes_' + todaysdate + '.csv'
stu2_secondtype_firsttypes.to_csv(spreadsheet)
files.download(spreadsheet)

*Do some calculations to make sure that the dates are reasonable (in case there was someone else with the same name on the BBS site, for instance), and get rid of all the junk in the sheet that is not needed.*

In [ ]:
stu2_secondtype_firsttypes['entry'] = "09/01/" + stu2_secondtype_firsttypes['entered']
stu2_secondtype_firsttypes['entryy'] = pd.to_datetime(stu2_secondtype_firsttypes['entry'])
stu2_secondtype_firsttypes['graddate'] = stu2_secondtype_firsttypes['entryy'] + timedelta(days=730)

# stu2_secondtype_firsttypes['firstdatetime'] = pd.to_datetime(stu2_secondtype_firsttypes['first_date']) ## start here with mft, pcc labels
# stu2_secondtype_firsttypes['seconddatetime'] = pd.to_datetime(stu2_secondtype_firsttypes['second_date'])

# def first_reasonable(c):
#     if (c['firstdatetime'] - c['graddate']) < timedelta(days=-500):
#         return 'NaN'
#     else:
#         return c['firstdatetime']
# stu2_secondtype_firsttypes['first_reasonable'] = stu2_secondtype_firsttypes.apply(first_reasonable, axis=1)

# def second_reasonable(c):
#     if (c['seconddatetime'] - c['graddate']) < timedelta(days=-500):
#         return 'NaN'
#     else:
#         return c['seconddatetime']
# stu2_secondtype_firsttypes['second_reasonable'] = stu2_secondtype_firsttypes.apply(second_reasonable, axis=1)

# stu2_secondtype_firsttypes

# studates = stu2_secondtype_firsttypes.filter(['NameLast','NameFirst','comboname','entryy','graddate','first_type','first_reasonable','second_type','second_reasonable'])

lic = {}
for a,b,i,j,z in zip(studates['first_type'],studates['first_reasonable'],studates['second_type'],studates['second_reasonable'],studates['comboname']):
    if a == 'Licensed ':
        lic_date = b
    elif i == 'Licensed ':
        lic_date = j
    else:
        lic_date = 'NaN'
    lic.update({z : lic_date})   
lic_dates = pd.DataFrame.from_dict(lic, orient='index', columns=['lic_date'])
studates_lic = studates.join(lic_dates, on='comboname', how='left', lsuffix='_left', rsuffix='_right')

assoc = {}
for a,b,i,j,z in zip(studates['first_type'],studates['first_reasonable'],studates['second_type'],studates['second_reasonable'],studates['comboname']):
    if a == 'Associate':
        assoc_date = b
    elif i == 'Associate':
        assoc_date = j
    else:
        assoc_date = 'NaN'
    assoc.update({z : assoc_date})   
assoc_dates = pd.DataFrame.from_dict(assoc, orient='index', columns=['assoc_date'])
studates_lic_assoc = studates_lic.join(assoc_dates, on='comboname', how='left', lsuffix='_left', rsuffix='_right')

justdates = studates_lic_assoc.filter(['NameLast','NameFirst','entryy','assoc_date','lic_date'])
justdates.head()

,NameLast,NameFirst,entryy,assoc_date,lic_date
0,BERMAN,CATHY,2006-09-01,2008-10-14,2013-04-30
1,LANKFORD,ASHBY,2006-09-01,2009-01-15,2013-04-15
2,ORTIZ,LAURA,2006-09-01,NaT,2018-11-30
3,MOK,IRVING,2006-09-01,2008-10-14,2010-10-14
4,KONIGSBERG,LAURA,2006-09-01,2008-12-08,NaT


*Pull all the people who got licensed in the last year and make a new spreadsheet. Download that spreadsheet so it can be used in Excel.*

In [ ]:
dateayrago = datetime.today() - pd.DateOffset(years=1)
lastyear = justdates['lic_date'] > dateayrago
liclastyear = justdates[lastyear].filter(['NameLast','NameFirst','lic_date'])
liclastyear.rename(columns={"NameLast": "Last name", "NameFirst": "First name", "lic_date": "MFT License issue date"}, inplace=True)
todaysdate = datetime.today().strftime('%m%d%Y')
filename = 'BBS_licensure_dates_for_last_year_' + todaysdate + '.csv'
liclastyear.to_csv(filename)
files.download(filename)
liclastyear.head()

,Last name,First name,License issue date
2,ORTIZ,LAURA,2018-11-30
90,AILSWORTH,SHIRLEY,2019-09-16
107,BROWN,JULIANE,2018-10-18
116,BRIANO,ALEXIS,2018-11-01
118,BROWN,VANESSA,2018-09-27


*Same thing, but this time we're exporting all the data (including the last year).*

In [ ]:
licall = justdates.filter(['NameLast','NameFirst','lic_date'])
licall.rename(columns={"NameLast": "Last name", "NameFirst": "First name", "lic_date": "MFT License issue date"}, inplace=True)
licall.dropna(inplace=True)
todaysdate = datetime.today().strftime('%m%d%Y')
filed = 'BBS_licensure_dates_all_data_' + todaysdate + '.csv'
licall.to_csv(filed)
filed.download(filed)
licall.head()

,Last name,First name,License issue date
0,BERMAN,CATHY,2013-04-30
1,LANKFORD,ASHBY,2013-04-15
2,ORTIZ,LAURA,2018-11-30
3,MOK,IRVING,2010-10-14
5,MEYER,AUDRIE,2010-06-16


In [ ]:
# Make sure full product name strings are included
# pd.set_option('max_colwidth', 75)

# import amazon purchases spreadsheet from google drive
spreadsheet = gc.open('BBS_licensure_dates_all_data_09242019')
sheet =  spreadsheet.get_worksheet(0)
everything = pd.DataFrame(sheet.get_all_records())
everything.rename(columns={"First name": "NameFirst", "Last name": "NameLast"}, inplace=True)
everything.head()

,,NameFirst,NameLast,License issue date
0,0,CATHY,BERMAN,2013-04-30
1,1,ASHBY,LANKFORD,2013-04-15
2,2,LAURA,ORTIZ,2018-11-30
3,3,IRVING,MOK,2010-10-14
4,5,AUDRIE,MEYER,2010-06-16


In [ ]:
ready = everything.merge(stus, on=['NameFirst','NameLast',])
ready['latlic'] = (pd.to_datetime(ready['License issue date']) - pd.to_datetime(ready['entered']) - pd.Timedelta(days=1000)) / pd.Timedelta(days=365)
fullydone = ready[ready['entered'].astype(int) <= 2010]
print(fullydone['latlic'].median())
fullydone.describe()

4.515068493150685


,,index,latlic
count,95.000000,95.000000,95.000000
mean,76.031579,267.852632,4.407585
std,45.745004,162.334955,1.570140
min,0.000000,0.000000,1.693151
25%,35.500000,122.000000,3.198630
50%,78.000000,268.000000,4.515068
75%,115.000000,404.500000,5.454795
max,154.000000,555.000000,10.180822


In [ ]:
todaysdate = datetime.today().strftime('%m%d%Y')
filed = 'fully_done_' + todaysdate + '.csv'
fullydone.to_csv(filed)
files.download(filed)